# Laboratorio 2 - Reglas de Asociación y Patrones Secuenciales

## Grupo 1 - Paula Hípola Gómez, José Ignacio Navas Sanz y Belén Ortega Pérez

## Entregable 1 - Reglas de Asociación

### 1.1 (1 punto) Estudia brevemente el dataset (número de usuarios, número de atributos, categorías más frecuentes para cada atributo/columna, etc.) y realiza el preprocesamiento para generar el dataset necesario para aplicar el algoritmo Apriori a este conjunto de datos.

In [1]:
%config IPCompleter.greedy=True
%reset
#Importamos las librerías necesarias
#Numpy
import numpy as np 
#Pandas
import pandas as pd
#Scikit-Learn
import sklearn as sk
#Matplotlib
import matplotlib.pyplot as plt
#Importamos la libreria mlxtend
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


### Cargamos los datos y los visualizamos

In [2]:
#Importamos el CSV
dataset = pd.read_csv('radio.csv')

#Mostramos los datos
dataset

,user,artist,sex,country
0,1,red hot chili peppers,f,Germany
1,1,the black dahlia murder,f,Germany
2,1,goldfrapp,f,Germany
3,1,dropkick murphys,f,Germany
4,1,le tigre,f,Germany
...,...,...,...,...
289950,19718,bob dylan,f,Canada
289951,19718,pixies,f,Canada
289952,19718,the clash,f,Canada
289953,19718,a tribe called quest,f,Canada


In [3]:
print('La dimensión del dataset es:',dataset.shape)

La dimensión del dataset es: (289955, 4)


Tenemos 289955 instancias y 4 atributos, para cada una de nuestras instancias. Analizando más profundamente el dataset, está compuesto con las interacciones de cada usuario que ha escuchado al menos a un artista. Vamos a ver cuantos valores tenemos actualmente en cuanto a usuarios, artistas y demás atributos y luego procederemos a identificar en el dataset valores duplicados y nulos que dificultan la comprensión de los datos y procederemos a su eliminación.

### Analizando los datos

In [4]:
#Valores desconocidos
desconocidos = dataset.loc[dataset['artist'] == '[unknown]']
print('Hay ', desconocidos.shape[0], 'valores desconocidos.');

Hay  553 valores desconocidos.


In [5]:
numUsuarios = dataset.groupby('user').count().shape[0]
print('Hay',numUsuarios,'usuarios.')
numArtistas = dataset.groupby('artist').count().shape[0]
print('Hay',numArtistas,'artistas.')
numSexos = dataset.groupby('sex').count().shape[0]
print('Hay',numSexos,'géneros.')
numPaises = dataset.groupby('country').count().shape[0]
print('Hay',numPaises,'países.')

Hay 15000 usuarios.
Hay 1004 artistas.
Hay 2 géneros.
Hay 159 países.


In [6]:
dataset['user'].describe()

count    289955.000000
mean       9852.460447
std        5692.355041
min           1.000000
25%        4935.000000
50%        9838.000000
75%       14769.000000
max       19718.000000
Name: user, dtype: float64

La función anterior nos muestra que el valor máximo para usuarios es de 19718 pero realmente hay 15000 usuarios. Esto se debe a que en la tabla tenemos las transacciones de aquellos que han escuchado al menos una canción pero se ha guardado su ID de registro. Es decir, cuando un Usuario se registra se le asigna un ID, por ejemplo el 1, pero si ese usuario no ha escuchado ninguna canción, no estará en nuestra base de datos. En cambio si el siguiente usuario con ID 4 escucha una canción si le tendremos en nuestro registro, de ahí que el valor máximo de usuarios sea 19718 pero realmente solo tenemos 15000, que son aquellos que si han escuchado al menos una canción.

In [7]:
dataset['artist'].describe()

count        289955
unique         1004
top       radiohead
freq           2704
Name: artist, dtype: object

Respecto a los artistas, antes hemos obtenido que había 1004 y efectivamente es lo que obtenemos cuando llamamos a la funcón 'describe'. Observamos que el artista más escuchado es radiohead, podremos analizar a este artista con respecto a los demás si tenemos algún interés especial en él.

In [8]:
dataset['sex'].describe()

count     289955
unique         2
top            m
freq      211823
Name: sex, dtype: object

Respecto al género, esta columna no nos aporta un valor diferencial. Simplemente nos indica si el usuario es mujer u hombre.

In [9]:
dataset['country'].describe()

count            289955
unique              159
top       United States
freq              59558
Name: country, dtype: object

Respecto al país puede jugar un valor algo más diferencial que el sexo en nuestro dataset pero consideramos que tampoco aporta un valor excesivamente diferencial puesto que nos vamos a centrar en los artistas

### Limpieza de datos

In [10]:
#Valores desconocidos
dataset.loc[dataset['artist'] == '[unknown]']
dataset = dataset.replace('[unknown]',np.NaN)

In [11]:
#Eliminación de nulos
dataset = dataset.dropna()
print('Tamaño sin nulos',dataset.shape)

#Eliminación de duplicados
dataset = dataset.drop_duplicates()
print('Tamaño sin duplicados',dataset.shape)
dataset

Tamaño sin nulos (289402, 4)
Tamaño sin duplicados (289400, 4)


,user,artist,sex,country
0,1,red hot chili peppers,f,Germany
1,1,the black dahlia murder,f,Germany
2,1,goldfrapp,f,Germany
3,1,dropkick murphys,f,Germany
4,1,le tigre,f,Germany
...,...,...,...,...
289950,19718,bob dylan,f,Canada
289951,19718,pixies,f,Canada
289952,19718,the clash,f,Canada
289953,19718,a tribe called quest,f,Canada


Tenemos que hacer un nuevo dataset para analizar estos datos con el algoritmo Apriori, para ello vamos a construir un dataset con un listado de ususarios con todos los artistas que ha escuchado...

In [12]:
dagrupado = dataset.groupby(['user','sex','country'])['artist'].agg(lambda x: list(x))
dfAgrupado = pd.DataFrame(dataset.groupby(['user','sex','country'],as_index=False)['artist'].agg(lambda x: list(x)))
dfAgrupado

,user,sex,country,artist
0,1,f,Germany,"[red hot chili peppers, the black dahlia murde..."
1,3,m,United States,"[devendra banhart, boards of canada, cocorosie..."
2,4,m,United Kingdom,"[tv on the radio, tool, kyuss, dj shadow, air,..."
3,5,m,Finland,"[dream theater, ac/dc, metallica, iron maiden,..."
4,6,m,Portugal,"[lily allen, kanye west, sigur rós, pink floyd..."
...,...,...,...,...
14990,19713,m,Ukraine,"[armin van buuren, above & beyond, atb, ferry ..."
14991,19714,m,United Kingdom,"[misfits, type o negative, arch enemy, red hot..."
14992,19715,m,United Kingdom,"[abba, james blunt, jason mraz, amy winehouse,..."
14993,19717,m,Brazil,"[marilyn manson, beyoncé, madonna, t.a.t.u., k..."


Pese a tener los datos del país del usuario además de su género, nos damos cuenta de que estos datos carecen de valor y no aportan nada al objetivo que perseguimos, que no es más que la adquisición de los conjuntos de grupos de música más frecuentes para la obtención de reglas de asociación que nos permita el estudio de estos grupos y sus similitudes por el cual se produzcan estos agrupamientos.

## 1.2 (4 puntos) Aprende a manejar la librería mlextend y su implementación del algoritmo Apriori.

In [13]:
te = TransactionEncoder()
te_ary = te.fit(dagrupado).transform(dagrupado)
df = pd.DataFrame(te_ary, columns=te.columns_)
df

,...and you will know us by the trail of dead,2pac,3 doors down,30 seconds to mars,311,36 crazyfists,44,50 cent,65daysofstatic,a day to remember,...,wilco,within temptation,wolfgang amadeus mozart,wu-tang clan,yann tiersen,yeah yeah yeahs,yellowcard,yo la tengo,zero 7,Édith piaf
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14990,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
14991,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
14992,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
14993,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


#### 1.2.1 Obtén los itemsets frecuentes para k=1. Para esto es necesario obtener el soporte de los itemset. Crea una función que utilice métodos de la librería mlextend de manera que dado un itemset devuelva su soporte.


Necesitamos definir un soporte mínimo. Sabemos que tenemos 14995 instancias, por lo que hemos considerado que aquellos valores que esten alrededor de 250/14995 ≈ 0.0167, serán válido para nosotros a la hora de operar con ello.

In [ ]:
frequent_itemsets = apriori(df, min_support=0.010, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets1 = frequent_itemsets[frequent_itemsets['length'] == 1]

In [ ]:
frequent_itemsets1.reset_index()

In [ ]:
artistas = [];
def soporteItemset(itemset):
    #Obtengo una lista de artistas
    for x in frequent_itemsets1['itemsets']:
        artista = (list(x))
        artistas.append(artista)
    #Recorro la lista de artistas
    for i in range(len(artistas)):
        if itemset in artistas[i]:
            soporte = frequent_itemsets1.loc[i]['support']
            return 'El sporte para ' + str(itemset) + ' es de: ' + str(round(soporte,4))
    return 'El artista ' + str(itemset) + ' no está en el itemset'

In [ ]:
coldplay = soporteItemset('coldplay')
print(coldplay)

ecdl = soporteItemset('el canto del loco')
print(ecdl)

beatles = soporteItemset('the beatles')
print(beatles)

#### 1.2.2. Crea una función que devuelva los itemsets frecuentes candidatos y su soporte para k>=2



Creamos una función que nos irá calculando itemsets frecuentes candidatos a medida que sea posible para k>=2 y que parará cuando no se generen estos itemsets. Esta función nos sirve además para el siguiente apartado donde se pide más o menos lo mismo

In [ ]:
def soporteItemsetKMayorIgual(k):
    #Creamos un diccionario. Esto servira para concatenarlo al nombre del itemset generado
    valoresValidosK = []
    data = dict();
    numCreados = 0;
    if k < 2:
        return 'La k debe ser mayor o igual a 2'
    else:
        while k != -1:
            print('Creando para k = ',k)
            frq = frequent_itemsets[frequent_itemsets['length'] == k]
            if(frq.shape[0] > 0):
                print(frq.shape[0])
                valoresValidosK.append(k);
                var = 'frq'+str(k);
                print(var,'Creado')
                print('Aumentamos k\n')
                k += 1;
                numCreados += 1;
            else:
                print('No se puede generar un itemset para k = ',k)
                print('Paramos el bucle. No generamos nada para k >= ',k)
                k = -1;
        for i in range(numCreados):
            data['frq'+str(valoresValidosK[i])] = frequent_itemsets[frequent_itemsets['length'] == valoresValidosK[i]]        
        print('\nPara obtener los itemsets creados consulte el diccionario que devuelve la funcion.')
        print('Se han generado',numCreados,'datasets')
        return data,numCreados

In [ ]:
itemsets, itemSetsGenerados = soporteItemsetKMayorIgual(2)

#### 1.2.3. Crea una función que repita el paso 1.2.2. hasta que no se generen nuevos itemsets frecuentes



La función que hemos generado en el paso 1.2.2 ya nos hace automáticamente esta función. Es decir, hemos creado una función que nos genera automáticamente los itemsets para k>=2 y que para cuando no hay itemsets frecuentes, por tanto no es necesario volver a repetir esta función

In [ ]:
k2 = pd.DataFrame.from_dict(itemsets['frq2'])
k2

In [ ]:
k3 = pd.DataFrame.from_dict(itemsets['frq3'])
k3

#### 1.2.4. Crea una función que muestre todas las reglas posibles con su confianza

En esta función determinamos la métrica de confianza, ya que esta evaluará si se tratan de reglas de interés o no para ser tenidas en cuenta. Para posteriormente poder hacer un filtrado de las reglas según la confianza deseada.

In [ ]:
def sacarReglas(itemsets):
    reglas = association_rules(itemsets, support_only=False, metric="confidence", min_threshold=0.015)
    reglas = reglas[['antecedents', 'consequents', 'antecedent support', 'consequent support', 'support', 'lift', 'confidence']]
    return reglas
sacarReglas(frequent_itemsets)

#### 1.2.5. Lista todas las reglas que sean de alta confianza (p.ej. con valor >=0.5)



In [ ]:
reglasAltaConfi = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)
reglasAltaConfi = reglasAltaConfi[['antecedents', 'consequents', 'confidence', 'lift']]

In [ ]:
reglasAltaConfi

#### 1.2.6. Crea funciones que permitan lo siguiente:
<ul>
    <li><b>* devolver todas las reglas que contengan un antecesor dado</b></li>
    <li><b>* devolver todas las reglas que tengan una confianza mayor (o igual) que un umbral mínimo dado</b></li>
    <li><b>* devolver todas las reglas que tengan un lift en una de las tres franjas de estudio (under 1, over 1, =1)</b></li>
</ul>


Para crear las siguientes funciones se emplea la función where(), que les da valores Nan a todas aquellas instancias que no cumplen la condición dada. Una vez hecho esto se eliminan del índice todos aquellos valores Nan, dejando unicamente aquellas reglas que si cumplan con esta condición.

In [ ]:
def reglasAntecesor(antecesor):
    antecedents = reglasAltaConfi['antecedents']
    index = antecedents.where([bool(antecesor in a) for a in list(antecedents)]).dropna().index
    return reglasAltaConfi.loc[index, :]

#### Reglas con un antecesor dado por parámetro

In [ ]:
reglasAntecesor('the killers')

In [ ]:
def reglasConfianza(confianza):
    confidence = reglasAltaConfi['confidence']
    index = confidence.where(confidence >= confianza).dropna().index
    return reglasAltaConfi.loc[index, :] 

#### Reglas con la confianza mayor o igual que un valor dado

In [ ]:
reglasConfianza(0.6)

In [ ]:
def reglasLift(lft):
    lift = reglasAltaConfi['lift']
    if lft == 1: 
        index = lift.where(lift == lft).dropna().index
    elif lft > 1:
        index = lift.where(lift > lft).dropna().index
    elif lft < 1:
        index = lift.where(lift < lft).dropna().index
    
    return reglasAltaConfi.loc[index, :]

#### Reglas con Lift dado

In [ ]:
reglasLift(0.9)

In [ ]:
reglasLift(1)

In [ ]:
reglasLift(1.1)

Al clasificar las reglas según su lift nos damos cuenta de que todas nuestras reglas tienen un lift mayor a 1, esto nos indica que tanto el antecedente como el consecuente aparecen juntos más de lo esperado, dándonos a entender que el antecedente tiene un efecto positivo sobre el consecuente de la regla.

#### 1.2.7. Utiliza al menos dos representaciones gráficas para mostrar las reglas obtenidas e interpretar los datos

Al no especificar qué reglas debemos mostrar nos hemos decantado por sacar, de las reglas generales la relación de la confianza con el soporte y ver cómo están relacionadas entre ellas

In [ ]:
reglasGenerales = sacarReglas(frequent_itemsets)
reglasGenerales

In [ ]:
soporte = reglasGenerales['support'].values
confianza = reglasGenerales['confidence'].values

Obtenemos todas las reglas generales. Ahora pondremos la relación entre soporte y confianza en una gráfica. Donde podemos ver que la mayor variación es en el soporte, ya que la confianza se situa mayoritariamente entre los valores 0.00 y 0.04.

In [ ]:
import random
import matplotlib.pyplot as plt
 
for i in range (len(soporte)):
    soporte[i] = soporte[i] + 0.0025 * (random.randint(1,10) - 5) 
    confianza[i] = confianza[i] + 0.0025 * (random.randint(1,10) - 5)

fig = plt.figure(figsize = (10,8), dpi=360)
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Confianza', fontsize = 10)
ax.set_ylabel('Soporte', fontsize = 10)
ax.set_title('Relación Soporte - Confianza Reglas Asociación', fontsize = 14)
ax.scatter(soporte, confianza, alpha=0.3, marker="o")
plt.show()

Por último, como tenemos las reglas generales, buscamos en las reglas generales aquellas que tengan como consecuente un grupo, en nuestro caso vamos a elegir el grupo Queen y veremos como se relacionan antecedentes y consecuentes, es decir, una relación de cómo interactúan los usarios que escuchan Queen. Cuando escuchan a qué artistas, luego escuchan a Queen.

In [ ]:
queen = reglasGenerales[reglasGenerales['consequents'].astype(str).str.lower().str.contains('queen')]
queen.reset_index()

In [ ]:
antecedentes = []
consecuentes = []
for t in queen['antecedents']:
    texto = str(t);
    texto = texto.replace("frozenset({", "")
    texto = texto.replace("})", "")
    antecedentes.append(texto)
    

for t in queen['consequents']:
    texto = str(t);
    texto = texto.replace("frozenset({", "")
    texto = texto.replace("})", "")
    consecuentes.append(texto)

In [ ]:
data = {'antecedentes': antecedentes, 
        'consecuentes': consecuentes} 
  
# Convert the dictionary into DataFrame 
queen = pd.DataFrame(data) 
queen

In [ ]:
import networkx as nx

fig = plt.figure(figsize = (20,10), dpi=360)
ax = fig.add_subplot(1,1,1)
ax.set_title('Artistas Consecuentes de Queen', fontsize = 20)
GA=nx.from_pandas_edgelist(queen, source='antecedentes', target='consecuentes')
nx.draw(GA,with_labels=True)
plt.rcParams['font.size'] = 10
plt.show()

#### Podemos observar que Queen está relacionado directamente con grupos como The Killers, The Who o incluso Pink Floyd. Tiene una relación curiosa con Led Zeppelin, mediante la que descubrimos que se une a Queens of the Stone Age descubriendo aquí una nueva relación entre Queen y estos últimos mediante Led Zeppelin. Ocurre lo mismo con The Beatles y una consecuente subconexión con Pink Floyd